## Builder

When piecewise object construction is complicated,
provide an API for doing it succinctly.

In [1]:
class HtmlElement:
    indent_size = 2
    
    def __init__(self, name='', text=''):
        self.text = text
        self.name = name
        self.elements = []
        
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')

        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')

        for e in self.elements:
            lines.append(e.__str(indent + 1))

        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)

    def __str__(self):
        return self.__str(0)
    
    @staticmethod
    def create(name):
        return HtmlBuilder(name)
        
        
class HtmlBuilder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.__root = HtmlElement(root_name)
        
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        return self
    
    def __str__(self):
        return str(self.__root)
    
    
    
builder = HtmlBuilder('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder:')
print(builder)

builder = HtmlElement.create('ul')
builder.add_child_fluent('li', 'hello').add_child('li', 'world')
print('\nFluent builder:')
print(builder)

Ordinary builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>

Fluent builder:
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


### Builder Facets

In [4]:
class Person:
    def __init__(self):
        print('Creating an instance of Person\n')
        # address
        self.street_address = None
        self.postcode = None
        self.city = None
        # employment info
        self.company_name = None
        self.position = None
        self.annual_income = None

    def __str__(self) -> str:
        return f'Address: {self.street_address}, {self.postcode}, {self.city}\n' +\
            f'Employed at {self.company_name} as a {self.postcode} earning {self.annual_income}'
    
    
class PersonBuilder:
    def __init__(self, person=Person()):
        self.person = person
        
    @property
    def works(self):
        return PersonJobBuilder(self.person)
    
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)
        
    def build(self):
        return self.person
    
    
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self, company_name):
        self.person.company_name = company_name
        return self
    
    def as_a(self, position):
        self.person.position = position
        return self
    
    def earning(self, anual_income):
        self.person.annual_income = anual_income
        return self


class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
    
    def at(self, street_address):
        self.street_address = street_address
        return self
    
    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self
    
    def in_city(self, city):
        self.person.city = city
        return self
    
    
pb = PersonBuilder()
person = pb\
    .lives\
        .at('123 London Road')\
        .in_city('London')\
        .with_postcode('SW12BC')\
    .works\
        .at('Fabrikam')\
        .as_a('Engineer')\
        .earning(123000)\
    .build()
print(person)

Creating an instance of Person

Address: None, SW12BC, London
Employed at Fabrikam as a SW12BC earning 123000


### Builder Inheritance

In [6]:
class Person:
    def __init__(self):
        self.name = None
        self.position = None
        self.date_of_birth = None
        
    def __str__(self):
        return f"{self.name} born on {self.date_of_birth} " + \
               f"works as {self.position}"
        
class PersonBuilder:
    def __init__(self):
        self.person = Person()
    
    def build(self):
        return self.person
    

class PersonInfoBuilder(PersonBuilder):
    def called(self, name):
        self.person.name = name
        return self
    
class PersonJobBuilder(PersonInfoBuilder):
    def work_as_a(self, position):
        self.person.position = position
        return self
    
class PersonBrithDateBuilder(PersonJobBuilder):
    def born(self, date_of_birth):
        self.person.date_of_birth = date_of_birth
        return self
        
pb = PersonBrithDateBuilder()
me = pb\
    .called('Eric')\
    .work_as_a('programer')\
    .born('14031985')\
    .build()

print(me)

Eric born on 14031985 works as programer
